# Using QuantumExpresso DFT simulations using nanoHUB web API

Install the nanohubRemote Api, it only works with Python 2.7, a patch to support Python 3.X is on its way

In [1]:
#Install nanohubRemote API
#!git clone https://github.com/bhaley/nanoHUB_remote

## Libraries not installed by default on SciServer Containers
The following libraries are not installed by default, they have to be installed before running the notebook the first time. Libraries can be installed by uncommenting the bellow lines



In [2]:
#!pip install hublib
#!pip install plotly
#!pip install qgrid
#!pip install nglview 

## Libraries required

* nanoHUB_remote contains basic functions to stablish communication with nanohub webApi
* hublib includes all graphical components that allow user to change paramenters from a GUI directly in the Jupyter notebook
* Ipython are the core libraries 
* plotly is a visualization library that renders data sequences in 2d and 3d representations
* ipywidgets is the standart library provided by Jupyter to create GUI
* zlib library helps to decompress some files encoded in Xml descriptors
* skimage is a image processin library usefull to 3d reconstructions (Marching cubes)
* numpy is a library to manipulate data

In [3]:
#access rappture tools & interactive matlab plots
import nanoHUB_remote
import hublib.rappture as rappture
import hublib.tool as tool
import hublib.ui as ui
from hublib.ui import Dropdown
from IPython.display import display
from IPython.display import clear_output
from IPython.core.display import display, HTML
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Figure
import plotly.figure_factory as FF
import plotly.graph_objs as go
import ipywidgets as widgets
import sys
import xml.etree.ElementTree as ET
import numpy as np
import matplotlib
from base64 import b64decode, b64encode
import zlib
from skimage import measure
init_notebook_mode(connected=True)

<IPython.core.display.Javascript object>

## Functions for data processing

This functions process output information from rappture ouput and extract the data
* get_data : given a xml branch (field) and a descriptor (container) it returns all children of the branch
* get_xy : given a xml branch (field) that contains a rappture curve, it extract information of the xy 2D points
* get_text : given a xml branch (field) that contains a rappture entity, it extract text labels
* plot_xy : reconstruct 2D rappture plots using plotly
* get_vtk : given a xml branch (field) that contains a rappture field/sequenve, it extract a vtk file
* plot_vtk : reconstruct 3D rappture surfaces using marching cubes algorithm and visualizing surfaces with plotly




In [4]:
def get_data(field, container):
  component = field.findall(container)
  return component

def get_xy(field, container):
  list_v = []
  component = get_data(field, container) 
  for obj in component:
    xy = obj.find("xy").text
    list_v.append(xy)
  return list_v

def get_text(obj, fields):
  objf = obj
  text = ''
  try:
    for field in fields:
        objf = objf.find(field)
    text = objf.text
  except: 
    text = ''    
  return text

def plot_xy(fields):  
  PLOT_FIG = []
  for field in fields:
    component = get_xy(field, 'component')
    label = get_text(field, ["about","label"])
    title = get_text(field, ["about","group"])
    xaxis = get_text(field, ["xaxis","label"])
    xunits = get_text(field, ["xaxis","units"])
    xscale = get_text(field, ["xaxis","scale"])
    if xscale == "":
        xscale = "linear"
    yaxis = get_text(field, ["yaxis","label"])
    yunits = get_text(field, ["yaxis","units"])
    yscale = get_text(field, ["yaxis","scale"])    
    if yscale == "":
        yscale = "linear"
    for obj in component:    
      xy = obj.strip()
      xy = np.array(xy.splitlines())
      xy = xy[(xy != '')]
      xy = [np.fromstring(xy[i], dtype=float, sep=" ") for i in range(len(xy))]
      xy = np.concatenate(xy)
      xy = xy.reshape(int(len(xy)/2),2)
      trace1 = go.Scatter(
        x = xy[:,0],
        y = xy[:,1],
        mode = 'lines',
        name = label,
        line = dict(
            shape='spline'
        )
      )
    PLOT_FIG.append(trace1)
    
    layout = go.Layout(
        title=title,
        xaxis=dict(
            title = xaxis + " [" + xunits + "]",
            type=xscale,
            autorange=True,
            exponentformat = "e",
        ),
        yaxis=dict(
            title = yaxis + " [" + yunits + "]",
            type=yscale,
            autorange=True,
            exponentformat = "e"
        ),
        legend=dict(orientation="h"),          
    )    
  fig = go.Figure(data=PLOT_FIG, layout=layout)
  return iplot(fig, show_link=False)

def get_vtk(component, container):
  list_v = []
  for obj in component:
    vtk = obj.find("vtk").text
    compressed_header = '@@RP-ENC:zb64\n'
    hlen = len(compressed_header)
    vtk = vtk[hlen:-1]
    datavtk = b64decode(vtk)
    list_v.append(zlib.decompress(datavtk, zlib.MAX_WBITS | 32))
  return list_v
  
def plot_vtk( component ):
  PLOT_ARRAY = []
  PLOT_LAYOUT = {}
  component = get_vtk(component , 'component')
  datavtk = component[0]
  npdata = np.array(datavtk.splitlines())
  dimensions = npdata[4].split()
  dimensions = [int(dimensions[v]) for v in range (1,len(dimensions)) ]
  spacing = npdata[5].split()
  spacing = [float(spacing[v]) for v in range (1,len(spacing)) ]
  origin = npdata[6].split()
  origin = [float(origin[v]) for v in range (1,len(origin)) ]
  #print dimensions, spacing, origin
  values = [np.fromstring(npdata[i], dtype=float, sep=" ") for i in range(10, len(npdata))]
  #print values
  NUM_CONTOURS = '5'
  list = np.concatenate(values)
  min_val = np.amin(list)
  max_val = np.amax(list)
  #print min_val
  #print max_val
  vol = list.reshape(dimensions[0],dimensions[1],dimensions[2])
  #print vol
  data = []
  num_iso = int(NUM_CONTOURS);
  viridis_cmap = matplotlib.cm.get_cmap('viridis')
  norm = matplotlib.colors.Normalize(vmin=min_val, vmax=max_val)
  show_colorbar = True
  colormap = []
  #plotly hack to include custom colorbars
  trace = go.Scatter3d(
    x = [0 for i in range(num_iso+2)],
    y = [0 for i in range(num_iso+2)],
    z = [0 for i in range(num_iso+2)],
    mode='markers',
    marker=dict(
      size=0.1,
      color=np.linspace(min_val, max_val, num=num_iso+2), # set color to an array/list of desired values
      colorscale='Viridis',
      opacity=1,
      colorbar=dict(
          title = "$\\Psi^2$",
          xanchor = "right",
          exponentformat = 'e'
      ), 
    ), 
  )    
  for value in np.linspace(min_val, max_val, num=num_iso+2):
    color_rgb = matplotlib.colors.colorConverter.to_rgb(viridis_cmap(norm(value)));
    colormap.append(color_rgb)
    if value > min_val and value < max_val:
      vertices, simplices = measure.marching_cubes_classic(vol, value, (spacing[2],spacing[1],spacing[0]), 'ascent')
      x,y,z = zip(*vertices)
      fig = FF.create_trisurf(x=z,
                              y=y, 
                              z=x, 
                              plot_edges=False,
                              colormap=[color_rgb,color_rgb],
                              simplices=simplices,
                              show_colorbar=True,
                              title="Isosurface")
      fig['data'][0]['opacity'] = 0.4
      show_colorbar = False
      data.append(fig['data'][0])

  datavtk = component[1]
  #print datavtk

  npdata = np.array(datavtk.splitlines())
  dimensions = npdata[4].split()
  dimensions = [int(dimensions[v]) for v in range (1,len(dimensions)) ]
  spacing = npdata[5].split()
  spacing = [float(spacing[v]) for v in range (1,len(spacing)) ]
  origin = npdata[6].split()
  origin = [float(origin[v]) for v in range (1,len(origin)) ]
  values = [np.fromstring(npdata[i], dtype=float, sep=" ") for i in range(10, len(npdata))]
  list = np.concatenate(values)
  vol = list.reshape(int(dimensions[0]),int(dimensions[1]),int(dimensions[2]))
  color_rgb = matplotlib.colors.colorConverter.to_rgb(viridis_cmap(norm(value)));
  colormap.append(color_rgb)
  #print spacing
  
  vertices, simplices = measure.marching_cubes_classic(vol, 0.5, (spacing[2],spacing[1],spacing[0]))
  x,y,z = zip(*vertices)
  fig = FF.create_trisurf(x=z,
                          y=y, 
                          z=x, 
                          plot_edges=False,
                          colormap=['rgb(100, 100, 100)','rgb(100, 100, 100)'],
                          simplices=simplices,
                          show_colorbar=False,
                          title="Isosurface"
                         )
  fig['data'][0]['opacity'] = 0.1
  show_colorbar = False
  data.append(fig['data'][0])


  data.append(trace)
  PLOT_ARRAY = data
  fig['layout']['showlegend'] = False
  max_axis = max([spacing[0]*dimensions[0], spacing[1]*dimensions[1], spacing[2]*dimensions[2]])
  min_axis = min(origin)
  fig['layout']['scene']['xaxis']['autorange']=False
  fig['layout']['scene']['xaxis']['range']=[0,max_axis]
  fig['layout']['scene']['yaxis']['autorange']=False
  fig['layout']['scene']['yaxis']['range']=[0,max_axis]
  fig['layout']['scene']['zaxis']['autorange']=False
  fig['layout']['scene']['zaxis']['range']=[0,max_axis]
  PLOT_LAYOUT= fig['layout']

  PLOT_FIG = Figure(data=PLOT_ARRAY, layout=PLOT_LAYOUT)
  iplot(PLOT_FIG, show_link=False)    
    

## GUI using hublib

The following lines create a graphical user interface in the jupyter Notebook to set parameters
* BUILD_WIDGETS : create hublib widgets based on the global dictionary input_table
* HANDLE_BUTTON : update the global dictionary input_table when the user interact with the widgets


In [7]:
input_table = {
    'Premade atomistic structure' : {
        'type' : 'String', 'value' : 'Si diamond'
    } ,
    'Structure type' : {
        'type' : 'Number', 'value' : '2'
    } ,
    'Density of States Calculations' : {
        'type' : 'String', 'value' : 'yes'
    } ,
    'Enable occupation options' : {
        'type' : 'String', 'value' : 'no'
    } ,
    'Occupation' : {
        'type' : 'String', 'value' : 'tetrahedra'
    } ,
    'Title of Run' : {
        'type' : 'String', 'value' : 'Silicon band structure'
    } ,
    'Atomic Mass' : {
        'type' : 'String', 'value' : 'Si 28.086'
    } ,
    'Atomic Coordinates' : {
        'type' : 'Number', 'value' : '0'
    } ,
    'energy grid step (eV)' : {
        'type' : 'Number', 'value' : '0.1'
    } ,
    'maximum Energy (eV)' : {
        'type' : 'Number', 'value' : '10'
    } ,
    'minimum Energy (eV)' : {
        'type' : 'Number', 'value' : '-6'
    } ,    
    'Band Structure Calculations' : {
        'type' : 'String', 'value' : 'yes'
    } ,        
    #'Phonon Calculations' : {
    #    'type' : 'String', 'value' : 'no'
    #} ,        
    'Smearing' : {
        'type' : 'Number', 'value' : '0'
    } ,        
    'X direction' : {
        'type' : 'Number', 'value' : '8'
    } ,        
    'Y direction' : {
        'type' : 'Number', 'value' : '8'
    } ,        
    'Z direction' : {
        'type' : 'Number', 'value' : '8'
    } ,          
}


def HANDLE_BUTTON( type_sel ):
    if type_sel['owner'].hub_widget.name in input_table:
        input_table[type_sel['owner'].hub_widget.name]['value'] = type_sel['new']
    
def BUILD_WIDGETS( ):
    global input_table
    tabs = []
    for key in input_table:
      input_desc = input_table[key]
      val = None
      if 'type' in input_desc:
        if input_desc['type'] == "Integer":
            val = ui.Integer(name=key,description="",value=input_desc['value'])
        elif input_desc['type'] == "Number":
            val = ui.Number(name=key,description="",value=input_desc['value'])            
        elif input_desc['type'] == "String":
            val = ui.String(name=key,description=key,value=input_desc['value'])            
        elif input_desc['type'] == "Dropdown":
            val = ui.Dropdown(name=key,description=key, options=input_desc['options'], value=input_desc['value'])    
        if val is not None:
            tabs.append(val)
            val.dd.hub_widget = val            
            val.dd.observe(HANDLE_BUTTON, "value", 'change')
    return tabs

display (ui.Form(BUILD_WIDGETS(), name = 'Specifications'))

## Running a simulation on Nanohub

Following lines create a simple button that execute all steps required to submit a simulation to nanohub and process its output. User has to click on "Run Tool" button to extract the input parameters from the GUI described above.

In [13]:
import time 
run_results = {'results_loaded' : False}
def RUN_TOOL(arg):
    global run_results
    run_results = {'results_loaded' : False}
    run_results = {}
    run_results['tool_name'] = 'dftqe'
    tool_inputs = {}
    for key in input_table:
        tool_inputs[key] = input_table[key]['value']
    run_results['tool_inputs'] = tool_inputs
    from mysecrets import auth_data        
    print ("Loading Nanohub Token..")
    run_results['headers'] = nanoHUB_remote.authenticate(auth_data)       
    print ("Loading XML Driver (this can take a few minutes) ...")
    run_results['driver_json'] = nanoHUB_remote.get_driver(run_results['tool_name'], run_results['tool_inputs'], run_results['headers'])
    
    #this option is not incosistent in the tool, Should be blancl instead of None / bug reported to developers
    run_results['driver_json']['xml'] = run_results['driver_json']['xml'].replace('<current>None</current>', '<current></current>')
    
    print ("Launching nanohub tool (this can take a few minutes)...")
    run_results['session_id'] = nanoHUB_remote.launch_tool(run_results['driver_json'], run_results['headers'])        
    print ("Loading Nanohub results for session ID:",run_results['session_id'], "..." )
    run_results['xml_results'] = nanoHUB_remote.get_results(run_results['session_id'], run_results['headers'])         
    print ("reconstructing XML tree ...",run_results['session_id'] )
    run_results['xml_tree'] = ET.fromstring(run_results['xml_results'])
    print ("extracting information from XML output ...")
    run_results['results'] = run_results['xml_tree'].find('output')
    print ("XML output save on run_results['results'] global variable" )
    run_results['results_loaded'] =  True
    
run_tool = widgets.Button(description='Run Tool')
run_tool.on_click(RUN_TOOL)
display(run_tool)


Button(description='Run Tool', style=ButtonStyle())

## Visualization of 2D curves
This code extract all the curves from the rappture output, creates a Dropdown with their labels and allow the user to visualize each curve independently

In [11]:
if (run_results['results_loaded'] == True):
    component = run_results['results'].findall("curve")
    key_component = {}
    for el in component:
        key_component[el.find("about").find("label").text] = el
    key = list(key_component.keys())
    def HANDLECURVES( type_sel ):
        global key_component
        clear_output()
        display(v)
        plot_xy([key_component[type_sel['new']]])

    v = Dropdown ("Output Curves", "", key, key[0] )
    v.dd.observe(HANDLECURVES, "value", 'change')
    display(v)
    HANDLECURVES( {'new':key[0]} )
else :
    print ("Results have not been loaded yet, make sure you already have clicked the 'Run Tool' button")

Box(children=(HTML(value='<p data-toggle="popover" title="">Output Curves</p>'), Dropdown(index=12, options=('…

## Visualization of multiple 2D curves
This code extract all the curves from the rappture output, creates multiple ToggleButtons and overlap all sets of data from selected curves

In [12]:
if (run_results['results_loaded'] == True):
    component = run_results['results'].findall("curve")
    key_component = {}
    for el in component:
        key_component[el.find("about").find("label").text] = el
    key = list(key_component.keys())
    v = []
        
    def HANDLEMULTCURVES( event ):
        global key_component
        clear_output()
        display(widgets.Box(v))
        list_comp = []
        for vv in v:
            if vv.value == True:
                list_comp.append(key_component[vv.description])
        if len(list_comp) > 0 :
            plot_xy(list_comp)
    for k in key:
        vv = widgets.ToggleButton(value=False, tooltip=k, description=k, icon='legal')
        vv.observe(HANDLEMULTCURVES, "value", 'change')
        v.append(vv)        
    display(widgets.Box(v))
    HANDLEMULTCURVES({})
else :
    print ("Results have not been loaded yet, make sure you already have clicked the 'Run Tool' button")

Box(children=(ToggleButton(value=False, description='Density of States', icon='legal', tooltip='Density of Sta…